In [6]:
import pandas as pd

In [7]:
csv_path = '../twitter_slo/slo_twitter_binary.csv'

def is_offensive(cls):
    if cls == 1: return 1
    else: return 0

def is_hate(cls):
    if cls == 0: return 1
    else: return 0

df = pd.read_csv(csv_path)
# df.drop(['count', 'neither', 'Unnamed: 0'], axis=1, inplace=True)
# df['tweet'].head()

In [ ]:
from preprocessing_utils import *

def clean_text(text):
    cleaned_text = re.sub(
        "(^!+)|([@][^\s]*[:]?)|(&#\d+;)|(https?[^\s]*)|(&amp;)|(RT)|(&lt;)|(\n)",
        "",
        text
    )

    # removing accented characters
    # cleaned_text = remove_accented_chars(cleaned_text)

    # removing special characters
    cleaned_text = remove_special_characters(cleaned_text)

    # removing contractions
    # cleaned_text = contractions.fix(cleaned_text)

    # removing punctuation
    cleaned_text = remove_punctuation(cleaned_text)

    # text stemming
    cleaned_text = stemming(cleaned_text, "slo")

    # removing extra whitespaces
    cleaned_text = cleaned_text.strip()
    cleaned_text = remove_extra_whitespace_tabs(cleaned_text)

    # print(text)
    # print(cleaned_text)
    # print()

    return cleaned_text

df['tweet'] = df['tweet'].apply(clean_text)
# df.head(30)
# df.to_csv('../twitter_slo/slo_twitter_binary_preprocessed.csv')